<H1>1) Data Load </H1>




1.1  Import All Spark Libraries 

In [1]:

import  org.apache.spark._ 
import  spark.implicits._
import org.apache.spark.sql.expressions.Window
import org.apache.spark.sql.SQLContext
import org.apache.spark.{SparkContext, SparkConf}
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import org.apache.spark.ml.feature.Imputer
import org.apache.spark.sql._
import SQLContext._





Intitializing Scala interpreter ...

Spark Web UI available at http://3724181074fa:4040
SparkContext available as 'sc' (version = 2.4.5, master = local[*], app id = local-1591271582616)
SparkSession available as 'spark'


2020-06-04 11:53:00,150 WARN  [main] util.NativeCodeLoader (NativeCodeLoader.java:<clinit>(62)) - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


import org.apache.spark._
import spark.implicits._
import org.apache.spark.sql.expressions.Window
import org.apache.spark.sql.SQLContext
import org.apache.spark.{SparkContext, SparkConf}
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import org.apache.spark.ml.feature.Imputer
import org.apache.spark.sql._
import SQLContext._


1.2 Read the Weather input file - this is stored in the local file directory 

In [2]:
val weather_org_df = spark.
        read.
        option("inferSchema", "true").
        option("header","true").
        option("mode","DROPMALFORMED").
        option("delimiter", ",").
        csv("weatherAUS.csv")

weather_org_df: org.apache.spark.sql.DataFrame = [Date: timestamp, Location: string ... 22 more fields]


1.3 Check the Schema of the Data frame

In [3]:
weather_org_df.printSchema()

root
 |-- Date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- MinTemp: string (nullable = true)
 |-- MaxTemp: string (nullable = true)
 |-- Rainfall: string (nullable = true)
 |-- Evaporation: string (nullable = true)
 |-- Sunshine: string (nullable = true)
 |-- WindGustDir: string (nullable = true)
 |-- WindGustSpeed: string (nullable = true)
 |-- WindDir9am: string (nullable = true)
 |-- WindDir3pm: string (nullable = true)
 |-- WindSpeed9am: string (nullable = true)
 |-- WindSpeed3pm: string (nullable = true)
 |-- Humidity9am: string (nullable = true)
 |-- Humidity3pm: string (nullable = true)
 |-- Pressure9am: string (nullable = true)
 |-- Pressure3pm: string (nullable = true)
 |-- Cloud9am: string (nullable = true)
 |-- Cloud3pm: string (nullable = true)
 |-- Temp9am: string (nullable = true)
 |-- Temp3pm: string (nullable = true)
 |-- RainToday: string (nullable = true)
 |-- RISK_MM: double (nullable = true)
 |-- RainTomorrow: string (nullable = true)



1.4 Transform the Dataframe columns to correct column type for effective use

In [4]:
val weather_org_df_cast = weather_org_df.withColumn("Date",to_date($"Date","dd/MM/yyyy"))
            .withColumn("MinTemp",col("MinTemp").cast(DoubleType))
            .withColumn("MaxTemp",col("MaxTemp").cast(DoubleType))
            .withColumn("Rainfall",col("Rainfall").cast(DoubleType))
            .withColumn("Evaporation",col("Evaporation").cast(DoubleType))
            .withColumn("Sunshine",col("Sunshine").cast(DoubleType))
            .withColumn("WindGustSpeed",col("WindGustSpeed").cast(DoubleType))
            .withColumn("WindSpeed9am",col("WindSpeed9am").cast(DoubleType))
            .withColumn("WindSpeed3pm",col("WindSpeed3pm").cast(DoubleType))
            .withColumn("Humidity9am",col("Humidity9am").cast(DoubleType))
            .withColumn("Humidity3pm",col("Humidity3pm").cast(DoubleType))
            .withColumn("Pressure9am",col("Pressure9am").cast(DoubleType))
            .withColumn("Pressure3pm",col("Pressure3pm").cast(DoubleType))
            .withColumn("Cloud9am",col("Cloud9am").cast(DoubleType))
            .withColumn("Cloud3pm",col("Cloud3pm").cast(DoubleType))
            .withColumn("Temp9am",col("Temp9am").cast(DoubleType))
            .withColumn("Temp3pm",col("Temp3pm").cast(DoubleType))
weather_org_df_cast.printSchema()

root
 |-- Date: date (nullable = true)
 |-- Location: string (nullable = true)
 |-- MinTemp: double (nullable = true)
 |-- MaxTemp: double (nullable = true)
 |-- Rainfall: double (nullable = true)
 |-- Evaporation: double (nullable = true)
 |-- Sunshine: double (nullable = true)
 |-- WindGustDir: string (nullable = true)
 |-- WindGustSpeed: double (nullable = true)
 |-- WindDir9am: string (nullable = true)
 |-- WindDir3pm: string (nullable = true)
 |-- WindSpeed9am: double (nullable = true)
 |-- WindSpeed3pm: double (nullable = true)
 |-- Humidity9am: double (nullable = true)
 |-- Humidity3pm: double (nullable = true)
 |-- Pressure9am: double (nullable = true)
 |-- Pressure3pm: double (nullable = true)
 |-- Cloud9am: double (nullable = true)
 |-- Cloud3pm: double (nullable = true)
 |-- Temp9am: double (nullable = true)
 |-- Temp3pm: double (nullable = true)
 |-- RainToday: string (nullable = true)
 |-- RISK_MM: double (nullable = true)
 |-- RainTomorrow: string (nullable = true)



weather_org_df_cast: org.apache.spark.sql.DataFrame = [Date: date, Location: string ... 22 more fields]


1.5 Basic checks to make sure all records are loaded from source 

In [5]:
weather_org_df_cast.count() // count of records for confirmation that all records are loaded

res2: Long = 142193


1.6 Create table for easy data manipulations

In [6]:
weather_org_df_cast.createOrReplaceTempView("Aus_Rain_Data")  // Create table for easy manipulations

In [7]:
spark.sql("select * from Aus_Rain_Data").show(5) // First look at the data

+----------+--------+-------+-------+--------+-----------+--------+-----------+-------------+----------+----------+------------+------------+-----------+-----------+-----------+-----------+--------+--------+-------+-------+---------+-------+------------+
|      Date|Location|MinTemp|MaxTemp|Rainfall|Evaporation|Sunshine|WindGustDir|WindGustSpeed|WindDir9am|WindDir3pm|WindSpeed9am|WindSpeed3pm|Humidity9am|Humidity3pm|Pressure9am|Pressure3pm|Cloud9am|Cloud3pm|Temp9am|Temp3pm|RainToday|RISK_MM|RainTomorrow|
+----------+--------+-------+-------+--------+-----------+--------+-----------+-------------+----------+----------+------------+------------+-----------+-----------+-----------+-----------+--------+--------+-------+-------+---------+-------+------------+
|2008-12-01|  Albury|   13.4|   22.9|     0.6|       null|    null|          W|         44.0|         W|       WNW|        20.0|        24.0|       71.0|       22.0|     1007.7|     1007.1|     8.0|    null|   16.9|   21.8|       No|  

# 2 Data Analysis 

2.1 Weather stations Data collected

In [8]:
val weather_dist_Location = weather_org_df_cast.select($"Location").groupBy("Location").count()
weather_dist_Location.count() // Number of Locations data collected

weather_dist_Location: org.apache.spark.sql.DataFrame = [Location: string, count: bigint]
res5: Long = 49


In [9]:
spark.sql("SELECT distinct location from Aus_Rain_Data order by Location").show(49) // Check the location of the weather stations

+----------------+
|        location|
+----------------+
|        Adelaide|
|          Albany|
|          Albury|
|    AliceSprings|
|   BadgerysCreek|
|        Ballarat|
|         Bendigo|
|        Brisbane|
|          Cairns|
|        Canberra|
|           Cobar|
|    CoffsHarbour|
|        Dartmoor|
|          Darwin|
|       GoldCoast|
|          Hobart|
|       Katherine|
|      Launceston|
|       Melbourne|
|MelbourneAirport|
|         Mildura|
|           Moree|
|    MountGambier|
|     MountGinini|
|       Newcastle|
|            Nhil|
|       NorahHead|
|   NorfolkIsland|
|       Nuriootpa|
|      PearceRAAF|
|         Penrith|
|           Perth|
|    PerthAirport|
|        Portland|
|        Richmond|
|            Sale|
|      SalmonGums|
|          Sydney|
|   SydneyAirport|
|      Townsville|
|     Tuggeranong|
|           Uluru|
|      WaggaWagga|
|         Walpole|
|        Watsonia|
|     Williamtown|
|     Witchcliffe|
|      Wollongong|
|         Woomera|
+-----------

<b>Data is available in 49 weather stations </b>

2.2 Number of Years The records are available

In [10]:

spark.sql("select date as First_record from Aus_Rain_Data order by date limit 1 ").show()
spark.sql("select date as Last_Record from Aus_Rain_Data order by date desc limit 1 ").show()




+------------+
|First_record|
+------------+
|  2007-11-01|
+------------+

+-----------+
|Last_Record|
+-----------+
| 2017-06-25|
+-----------+



<b>Data is availble for from Nov 2007 to June 2017 - Nearly 10 years records . </b>

2.3 Data Range across Data

In [11]:
weather_org_df_cast.select("MinTemp","MaxTemp","Temp9am","Temp3pm","Pressure9am","Pressure3pm").describe().show()
weather_org_df_cast.select("WindGustSpeed","WindSpeed9am","WindSpeed3pm","Rainfall","Evaporation","Sunshine").describe().show()
weather_org_df_cast.select("Cloud9am","Cloud3pm","Humidity9am","Humidity3pm").describe().show()

+-------+------------------+-----------------+-----------------+-----------------+------------------+-----------------+
|summary|           MinTemp|          MaxTemp|          Temp9am|          Temp3pm|       Pressure9am|      Pressure3pm|
+-------+------------------+-----------------+-----------------+-----------------+------------------+-----------------+
|  count|            141556|           141871|           141289|           139467|            128179|           128212|
|   mean|12.186399728729263|23.22678419127248|16.98750858170129|21.68723497314778|1017.6537584159663|1015.258203537901|
| stddev| 6.403282674671343|7.117618141018147|6.492838325478932|6.937593868533731|7.1054757115203016|7.036676783493396|
|    min|              -8.5|             -4.8|             -7.2|             -5.4|             980.5|            977.1|
|    max|              33.9|             48.1|             40.2|             46.7|            1041.0|           1039.6|
+-------+------------------+------------

<b>Data across the entire population of data might give wrong interpretation . Lets look at by location as Australia is very big country and weather across different States vary too much</b>

2.4 Data Range across Different Location

In [12]:
// Partition the data to get by location & Date

val winp1 = Window.partitionBy("Location").orderBy(col("Date"))
    weather_org_df.withColumn("row",row_number.over(winp1))
      .where($"row" === 1).drop("row")

val winp2 = Window.partitionBy("Location")
    val aggDF = weather_org_df.withColumn("row",row_number.over(winp1))
    .withColumn("MinTempavg", avg(col("MinTemp")).over(winp2))
    .withColumn("MaxTempavg", avg(col("MaxTemp")).over(winp2))
    .withColumn("Rfavg", avg(col("Rainfall")).over(winp2))
    .withColumn("Rfavg", avg(col("Rainfall")).over(winp2))
    .withColumn("WindGSavg", avg(col("WindGustSpeed")).over(winp2))
    
      .where(col("row")===1).select("Location","MinTempavg","MaxTempavg","Rfavg","WindGSavg")
      .orderBy("Location").show(49)  // show all Weather station locations
   

2020-06-04 11:53:55,677 WARN  [Thread-4] util.Utils (Logging.scala:logWarning(66)) - Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.debug.maxToStringFields' in SparkEnv.conf.
+----------------+------------------+------------------+------------------+------------------+
|        Location|        MinTempavg|        MaxTempavg|             Rfavg|         WindGSavg|
+----------------+------------------+------------------+------------------+------------------+
|        Adelaide|12.628367875647646|22.945401554404082| 1.572185430463575| 36.53081186827519|
|          Albany|12.948461278322574|20.072587131367285| 2.255073431241649|              null|
|          Albury|  9.52089850249585|22.630963455149466| 1.925710464727508| 32.95301566144619|
|    AliceSprings|13.125181518151814|29.244191419141917|0.8693553719008265| 40.53371351895337|
|   BadgerysCreek|11.136899862825791|24.023111111111124|2.2079249217935253| 33.6098901098

winp1: org.apache.spark.sql.expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@17781a6f
winp2: org.apache.spark.sql.expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@21588061
aggDF: Unit = ()


<b>As expected, We can see that there is a wide difference in the Weather conditions across , for example In Tuggeranog station the Average Min Tem is 7.24 but the average in Darwin is 23.2 </b>

2.5 Check the Null / NA columns acorss the data Range to decide on the data wrangling requirement 

In [13]:
// Checking null Values across all acolumns at once !!! 

weather_org_df_cast.select(weather_org_df_cast.columns.map(colName => {
    count(when(col(colName).isNull, true) || when(col(colName)==="", true)) as s"${colName}"
  }): _*)
  .show()

+----+--------+-------+-------+--------+-----------+--------+-----------+-------------+----------+----------+------------+------------+-----------+-----------+-----------+-----------+--------+--------+-------+-------+---------+-------+------------+
|Date|Location|MinTemp|MaxTemp|Rainfall|Evaporation|Sunshine|WindGustDir|WindGustSpeed|WindDir9am|WindDir3pm|WindSpeed9am|WindSpeed3pm|Humidity9am|Humidity3pm|Pressure9am|Pressure3pm|Cloud9am|Cloud3pm|Temp9am|Temp3pm|RainToday|RISK_MM|RainTomorrow|
+----+--------+-------+-------+--------+-----------+--------+-----------+-------------+----------+----------+------------+------------+-----------+-----------+-----------+-----------+--------+--------+-------+-------+---------+-------+------------+
|   0|       0|    637|    322|    1406|      60843|   67816|          0|         9270|         0|         0|        1348|        2630|       1774|       3610|      14014|      13981|   53657|   57094|    904|   2726|        0|      0|           0|
+---

<b>There is significantly large missing data in Evaporation , Sunshine and Cloud Data set . In the next feature during the Feature selection for modeling we need to make note of this.</b>

<H1>3) Data Analysis using visulaisation   </H1>

Our team decided to use Python for this purpose . A separate Python files has been attached and screen shots of those are attached to the Report

<H1>4) Data Wrangling  </H1>

In [14]:
// Remove duplicates based on Date and Location

val weather_org_df_W1 =  weather_org_df_cast.distinct()
weather_org_df_W1.count()


weather_org_df_W1: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Date: date, Location: string ... 22 more fields]
res10: Long = 142193


In [15]:
val weather_org_df_W2 =  weather_org_df_cast.dropDuplicates(Seq("Date", "Location"))
weather_org_df_W2.count()

weather_org_df_W2: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Date: date, Location: string ... 22 more fields]
res11: Long = 142193


4.2  Convert blank columns into uniform notation so that imputation can be applied as uniform. As the data set is a time series data , imputing value will value can be  based 

    a) Regression
    b) Nearest Data set approach
    c) Splines (using MARS technique)
    d) Simplying imputing avaerage of that column (This wil not be correct as the weather data is wide spread across States)
    
    We have follwed nearest data apprach as it is one of the most suited 



In [16]:
weather_org_df_cast.filter($"MinTemp".isNull).count()

res12: Long = 637


In [17]:
val weather_org_df_W2_impute= weather_org_df_cast.withColumn("MinTemp",when(col("MinTemp").isNull,lead("MinTemp",1).over(winp1)).otherwise(col("MinTemp")))
                              .withColumn("MaxTemp",when(col("MaxTemp").isNull,lead("MaxTemp",1).over(winp1)).otherwise(col("MaxTemp")))
                             .withColumn("Temp9am",when(col("Temp9am").isNull,lead("Temp9am",1).over(winp1)).otherwise(col("Temp9am")))
                             .withColumn("Temp3pm",when(col("Temp3pm").isNull,lead("Temp3pm",1).over(winp1)).otherwise(col("Temp3pm")))
                              .withColumn("Cloud9am",when(col("Cloud9am").isNull,lead("Cloud9am",1).over(winp1)).otherwise(col("Cloud9am")))
                            .withColumn("Cloud3pm",when(col("Cloud3pm").isNull,lead("Cloud3pm",1).over(winp1)).otherwise(col("Cloud3pm")))
                            .withColumn("Cloud3pm",when(col("Cloud3pm").isNull,lead("Cloud3pm",1).over(winp1)).otherwise(col("Cloud3pm")))
                            .withColumn("Humidity9am",when(col("Humidity9am").isNull,lead("Humidity9am",1).over(winp1)).otherwise(col("Humidity9am")))
                            .withColumn("Humidity3pm",when(col("Humidity3pm").isNull,lead("Humidity3pm",1).over(winp1)).otherwise(col("Humidity3pm")))
                            .withColumn("Pressure9am",when(col("Pressure9am").isNull,lead("Pressure9am",1).over(winp1)).otherwise(col("Pressure9am")))
                             .withColumn("Pressure9am",when(col("Pressure9am").isNull,lead("Pressure9am",1).over(winp1)).otherwise(col("Pressure9am")))
                            .withColumn("Pressure3pm",when(col("Pressure3pm").isNull,lead("Pressure3pm",1).over(winp1)).otherwise(col("Pressure3pm")))
                             .withColumn("WindGustSpeed",when(col("WindGustSpeed").isNull,lead("WindGustSpeed",1).over(winp1)).otherwise(col("WindGustSpeed")))
                            .withColumn("WindSpeed9am",when(col("WindSpeed9am").isNull,lead("WindSpeed9am",1).over(winp1)).otherwise(col("WindSpeed9am")))
                             .withColumn("WindSpeed3pm",when(col("WindSpeed3pm").isNull,lead("WindSpeed3pm",1).over(winp1)).otherwise(col("WindSpeed3pm")))
                             .withColumn("Rainfall",when(col("Rainfall").isNull,lead("Rainfall",1).over(winp1)).otherwise(col("Rainfall")))
                             .withColumn("Evaporation",when(col("Evaporation").isNull,lead("Evaporation",1).over(winp1)).otherwise(col("Evaporation")))
                             .withColumn("Sunshine",when(col("Sunshine").isNull,lead("Sunshine",1).over(winp1)).otherwise(col("Sunshine")))

weather_org_df_W2_impute: org.apache.spark.sql.DataFrame = [Date: date, Location: string ... 22 more fields]


In [18]:
weather_org_df_W2_impute.createOrReplaceTempView("Aus_Rain_Data1")  // Create table for easy manipulations

Test Imputation 

In [19]:
spark.sql("select COUNT(*) from Aus_Rain_Data1 where MinTemp IS NULL ").show()
spark.sql("select COUNT(*) from Aus_Rain_Data where MinTemp IS NULL ").show()

+--------+
|count(1)|
+--------+
|     164|
+--------+

+--------+
|count(1)|
+--------+
|     637|
+--------+



In [20]:
val weather_org_df_W3_impute= weather_org_df_W2_impute.withColumn("MinTemp",when(col("MinTemp").isNull,lag("MinTemp",1).over(winp1)).otherwise(col("MinTemp")))
                              .withColumn("MaxTemp",when(col("MaxTemp").isNull,lag("MaxTemp",1).over(winp1)).otherwise(col("MaxTemp")))
                             .withColumn("Temp9am",when(col("Temp9am").isNull,lag("Temp9am",1).over(winp1)).otherwise(col("Temp9am")))
                             .withColumn("Temp3pm",when(col("Temp3pm").isNull,lag("Temp3pm",1).over(winp1)).otherwise(col("Temp3pm")))
                              .withColumn("Cloud9am",when(col("Cloud9am").isNull,lag("Cloud9am",1).over(winp1)).otherwise(col("Cloud9am")))
                            .withColumn("Cloud3pm",when(col("Cloud3pm").isNull,lag("Cloud3pm",1).over(winp1)).otherwise(col("Cloud3pm")))
                            .withColumn("Cloud3pm",when(col("Cloud3pm").isNull,lag("Cloud3pm",1).over(winp1)).otherwise(col("Cloud3pm")))
                            .withColumn("Humidity9am",when(col("Humidity9am").isNull,lag("Humidity9am",1).over(winp1)).otherwise(col("Humidity9am")))
                            .withColumn("Humidity3pm",when(col("Humidity3pm").isNull,lag("Humidity3pm",1).over(winp1)).otherwise(col("Humidity3pm")))
                            .withColumn("Pressure9am",when(col("Pressure9am").isNull,lag("Pressure9am",1).over(winp1)).otherwise(col("Pressure9am")))
                             .withColumn("Pressure9am",when(col("Pressure9am").isNull,lag("Pressure9am",1).over(winp1)).otherwise(col("Pressure9am")))
                            .withColumn("Pressure3pm",when(col("Pressure3pm").isNull,lag("Pressure3pm",1).over(winp1)).otherwise(col("Pressure3pm")))
                             .withColumn("WindGustSpeed",when(col("WindGustSpeed").isNull,lag("WindGustSpeed",1).over(winp1)).otherwise(col("WindGustSpeed")))
                            .withColumn("WindSpeed9am",when(col("WindSpeed9am").isNull,lag("WindSpeed9am",1).over(winp1)).otherwise(col("WindSpeed9am")))
                             .withColumn("WindSpeed3pm",when(col("WindSpeed3pm").isNull,lag("WindSpeed3pm",1).over(winp1)).otherwise(col("WindSpeed3pm")))
                             .withColumn("Rainfall",when(col("Rainfall").isNull,lag("Rainfall",1).over(winp1)).otherwise(col("Rainfall")))
                             .withColumn("Evaporation",when(col("Evaporation").isNull,lag("Evaporation",1).over(winp1)).otherwise(col("Evaporation")))
                             .withColumn("Sunshine",when(col("Sunshine").isNull,lag("Sunshine",1).over(winp1)).otherwise(col("Sunshine")))

weather_org_df_W3_impute: org.apache.spark.sql.DataFrame = [Date: date, Location: string ... 22 more fields]


In [21]:
weather_org_df_W3_impute.createOrReplaceTempView("Aus_Rain_Data2")

In [22]:
spark.sql("select COUNT(*) from Aus_Rain_Data2 where MinTemp IS NULL ").show()


+--------+
|count(1)|
+--------+
|     102|
+--------+



<b> For Remaining Null Values, Lets impute using the Monthly average for that location</b>

In [23]:
// First lets create a Month column to partition the records by Month and Location

val weather_org_df_W4_impute = weather_org_df_W3_impute.withColumn("Month",month(col("Date")))

weather_org_df_W4_impute: org.apache.spark.sql.DataFrame = [Date: date, Location: string ... 23 more fields]


In [24]:
weather_org_df_W4_impute.printSchema()

root
 |-- Date: date (nullable = true)
 |-- Location: string (nullable = true)
 |-- MinTemp: double (nullable = true)
 |-- MaxTemp: double (nullable = true)
 |-- Rainfall: double (nullable = true)
 |-- Evaporation: double (nullable = true)
 |-- Sunshine: double (nullable = true)
 |-- WindGustDir: string (nullable = true)
 |-- WindGustSpeed: double (nullable = true)
 |-- WindDir9am: string (nullable = true)
 |-- WindDir3pm: string (nullable = true)
 |-- WindSpeed9am: double (nullable = true)
 |-- WindSpeed3pm: double (nullable = true)
 |-- Humidity9am: double (nullable = true)
 |-- Humidity3pm: double (nullable = true)
 |-- Pressure9am: double (nullable = true)
 |-- Pressure3pm: double (nullable = true)
 |-- Cloud9am: double (nullable = true)
 |-- Cloud3pm: double (nullable = true)
 |-- Temp9am: double (nullable = true)
 |-- Temp3pm: double (nullable = true)
 |-- RainToday: string (nullable = true)
 |-- RISK_MM: double (nullable = true)
 |-- RainTomorrow: string (nullable = true)
 |-- M

<b> Month Column is added to the data frame </b>

First lets check the Monthly Averages before we impute

In [25]:
val winp3 = Window.partitionBy("Location","Month").orderBy(col("Date"))
    weather_org_df_W4_impute.withColumn("row",row_number.over(winp3))
      .where($"row" === 1).drop("row")

val winp4 = Window.partitionBy("Location","Month")
    val aggDF = weather_org_df_W4_impute.withColumn("row",row_number.over(winp3))
    .withColumn("MinTempavg", avg(col("MinTemp")).over(winp4))
    .withColumn("MaxTempavg", avg(col("MaxTemp")).over(winp4))
    .withColumn("Rfavg", avg(col("Rainfall")).over(winp4))
    .withColumn("Evapavg", avg(col("Evaporation")).over(winp4))
    .withColumn("Sunavg", avg(col("Sunshine")).over(winp4))
    .withColumn("WindGusavg", avg(col("WindGustSpeed")).over(winp4))
    .withColumn("WindS9amavg", avg(col("WindSpeed9am")).over(winp4))
    .withColumn("WindS3pmavg", avg(col("WindSpeed3pm")).over(winp4))
    .withColumn("Hum9amavg", avg(col("Humidity9am")).over(winp4))
    .withColumn("Hum3pmavg", avg(col("Humidity3pm")).over(winp4))
    .withColumn("Pres9amavg", avg(col("Pressure9am")).over(winp4))
    .withColumn("Pres3pmavg", avg(col("Pressure3pm")).over(winp4))
    .withColumn("Clo9amavg", avg(col("Cloud9am")).over(winp4))
    .withColumn("Clo9pmavg", avg(col("Cloud3pm")).over(winp4))
    .withColumn("Tem9amavg", avg(col("Temp9am")).over(winp4))
    .withColumn("Tem3pmavg", avg(col("Temp3pm")).over(winp4))
    
      .where(col("row")===1).select("Month","Location","MinTempavg","MaxTempavg","Rfavg","Evapavg","Sunavg","WindGusavg","WindS9amavg","WindS3pmavg","Hum9amavg",
                                    "Hum3pmavg","Pressure9am","Pressure3pm","Cloud9am","Cloud3pm","Temp9am","Temp3pm")
      .orderBy("Location","Month").toDF()  // 

winp3: org.apache.spark.sql.expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@cd18bdd
winp4: org.apache.spark.sql.expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@5d2432f5
aggDF: org.apache.spark.sql.DataFrame = [Month: int, Location: string ... 16 more fields]


Lets check the averages before impute 

In [26]:
aggDF.createOrReplaceTempView("Aus_Rain_Data_testing")

In [27]:
spark.sql("select * from Aus_Rain_Data_testing where Location = 'Sydney'" ).show()

+-----+--------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-----------+-----------+--------+--------+-------+-------+
|Month|Location|        MinTempavg|        MaxTempavg|             Rfavg|           Evapavg|            Sunavg|        WindGusavg|       WindS9amavg|       WindS3pmavg|         Hum9amavg|         Hum3pmavg|Pressure9am|Pressure3pm|Cloud9am|Cloud3pm|Temp9am|Temp3pm|
+-----+--------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-----------+-----------+--------+--------+-------+-------+
|    1|  Sydney|20.316487455197134|27.517204301075285|3.1275985663082424| 7.705017921146956|7.5673835125448035|43.903225806451616| 12.46594982078853|21.205035971223023| 67.59498207885305|58.473118279569896

Averages are calculated correctly for each month 

In [28]:
val winp5 = Window.partitionBy("Location","Month").orderBy(col("Date"))
    weather_org_df_W4_impute.withColumn("row",row_number.over(winp3))
      .where($"row" === 1).drop("row")
val winp6 = Window.partitionBy("Location","Month")

val weather_org_df_ff= weather_org_df_W4_impute.withColumn("row",row_number.over(winp5))
.withColumn("MinTemp",when(col("MinTemp").isNull,avg(col("MinTemp")).over(winp6)).otherwise(col("MinTemp")))
.withColumn("MaxTemp",when(col("MaxTemp").isNull,avg(col("MaxTemp")).over(winp6)).otherwise(col("MaxTemp")))
.withColumn("Temp9am",when(col("Temp9am").isNull,avg(col("Temp9am")).over(winp6)).otherwise(col("Temp9am")))
.withColumn("Temp3pm",when(col("Temp3pm").isNull,avg(col("Temp3pm")).over(winp6)).otherwise(col("Temp3pm")))
.withColumn("Cloud9am",when(col("Cloud9am").isNull,avg(col("Cloud9am")).over(winp6)).otherwise(col("Cloud9am")))
.withColumn("Cloud3pm",when(col("Cloud3pm").isNull,avg(col("Cloud3pm")).over(winp6)).otherwise(col("Cloud3pm")))
.withColumn("Cloud3pm",when(col("Cloud3pm").isNull,avg(col("Cloud3pm")).over(winp6)).otherwise(col("Cloud3pm")))
.withColumn("Humidity9am",when(col("Humidity9am").isNull,avg(col("Humidity9am")).over(winp6)).otherwise(col("Humidity9am")))
.withColumn("Humidity3pm",when(col("Humidity3pm").isNull,avg(col("Humidity3pm")).over(winp6)).otherwise(col("Humidity3pm")))
.withColumn("Pressure9am",when(col("Pressure9am").isNull,avg(col("Pressure9am")).over(winp6)).otherwise(col("Pressure9am")))
.withColumn("Pressure9am",when(col("Pressure9am").isNull,avg(col("Pressure9am")).over(winp6)).otherwise(col("Pressure9am")))
.withColumn("Pressure3pm",when(col("Pressure3pm").isNull,avg(col("Pressure3pm")).over(winp6)).otherwise(col("Pressure3pm")))
.withColumn("WindGustSpeed",when(col("WindGustSpeed").isNull,avg(col("WindGustSpeed")).over(winp6)).otherwise(col("WindGustSpeed")))
.withColumn("WindSpeed9am",when(col("WindSpeed9am").isNull,avg(col("WindSpeed9am")).over(winp6)).otherwise(col("WindSpeed9am")))
.withColumn("WindSpeed3pm",when(col("WindSpeed3pm").isNull,avg(col("WindSpeed3pm")).over(winp6)).otherwise(col("WindSpeed3pm")))
.withColumn("Rainfall",when(col("Rainfall").isNull,avg(col("Rainfall")).over(winp6)).otherwise(col("Rainfall")))
.withColumn("Evaporation",when(col("Evaporation").isNull,avg(col("Evaporation")).over(winp6)).otherwise(col("Evaporation")))
.withColumn("Sunshine",when(col("Sunshine").isNull,avg(col("Sunshine")).over(winp6)).otherwise(col("Sunshine")))
.where(col("row")===1).select("Date","Month","Location","MinTemp","MaxTemp","Rainfall","Evaporation","Sunshine","WindGustDir",
                              "WindGustSpeed","WindDir9am","WindDir3pm",
                                    "WindSpeed9am","WindSpeed3pm","Humidity9am","Humidity3pm","Pressure9am","Pressure3pm",
                              "Cloud9am","Cloud3pm","Temp9am","Temp3pm","RainToday","RISK_MM","RainTomorrow")
      .orderBy("Location","Month").toDF()  

winp5: org.apache.spark.sql.expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@7f234df9
winp6: org.apache.spark.sql.expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@760ee998
weather_org_df_ff: org.apache.spark.sql.DataFrame = [Date: date, Month: int ... 23 more fields]


In [29]:
weather_org_df_ff.createOrReplaceTempView("Aus_Rain_Data_Final")

In [30]:
spark.sql("select * from Aus_Rain_Data_Final").show()

+----------+-----+--------+-------+-------+--------+-----------------+--------+-----------+-------------+----------+----------+------------+------------+-----------+-----------+-----------+-----------+--------+--------+-------+-------+---------+-------+------------+
|      Date|Month|Location|MinTemp|MaxTemp|Rainfall|      Evaporation|Sunshine|WindGustDir|WindGustSpeed|WindDir9am|WindDir3pm|WindSpeed9am|WindSpeed3pm|Humidity9am|Humidity3pm|Pressure9am|Pressure3pm|Cloud9am|Cloud3pm|Temp9am|Temp3pm|RainToday|RISK_MM|RainTomorrow|
+----------+-----+--------+-------+-------+--------+-----------------+--------+-----------+-------------+----------+----------+------------+------------+-----------+-----------+-----------+-----------+--------+--------+-------+-------+---------+-------+------------+
|2009-01-01|    1|Adelaide|   16.9|   22.9|     0.0|             14.2|    10.9|         SW|         50.0|        SW|        SW|        17.0|        20.0|       58.0|       40.0|     1011.3|     1012.

In [31]:
weather_org_df_ff.select(weather_org_df_cast.columns.map(colName => {
    count(when(col(colName).isNull, true) || when(col(colName)==="", true)) as s"${colName}"
  }): _*)
  .show()

+----+--------+-------+-------+--------+-----------+--------+-----------+-------------+----------+----------+------------+------------+-----------+-----------+-----------+-----------+--------+--------+-------+-------+---------+-------+------------+
|Date|Location|MinTemp|MaxTemp|Rainfall|Evaporation|Sunshine|WindGustDir|WindGustSpeed|WindDir9am|WindDir3pm|WindSpeed9am|WindSpeed3pm|Humidity9am|Humidity3pm|Pressure9am|Pressure3pm|Cloud9am|Cloud3pm|Temp9am|Temp3pm|RainToday|RISK_MM|RainTomorrow|
+----+--------+-------+-------+--------+-----------+--------+-----------+-------------+----------+----------+------------+------------+-----------+-----------+-----------+-----------+--------+--------+-------+-------+---------+-------+------------+
|   0|       0|      0|      0|       0|        197|     228|          0|           24|         0|         0|           0|           0|          0|          0|         48|         48|     144|     144|      0|      0|        0|      0|           0|
+---

Manage to remove null values for most of the columns . For some of the columns Like sunshine and Evaporation missing values still exist as the values are missing for the entire month. We will handle this if necessary during the modeling

4.3  Removing Data set out side of the expected range ( Excluding Outliers)

In [32]:
val weather_org_df_ff_outl = weather_org_df_ff.withColumn("MinTemp",when(col("MinTemp")<(-10),null).otherwise(col("MinTemp")))
            .withColumn("MinTemp",when(col("MinTemp")>(50),null).otherwise(col("MinTemp")))
            .withColumn("MaxTemp",when(col("MaxTemp")<(-10),null).otherwise(col("MaxTemp")))
            .withColumn("MaxTemp",when(col("MaxTemp")>(50),null).otherwise(col("MaxTemp")))
            .withColumn("Temp9am",when(col("Temp9am")<(-10),null).otherwise(col("Temp9am")))
            .withColumn("Temp9am",when(col("Temp9am")>(50),null).otherwise(col("Temp9am")))
            .withColumn("Temp3pm",when(col("Temp3pm")<(-10),null).otherwise(col("Temp3pm")))
            .withColumn("Temp3pm",when(col("Temp3pm")>(50),null).otherwise(col("Temp3pm")))
            .withColumn("Cloud9am",when(col("Cloud9am")<(0),null).otherwise(col("Cloud9am")))
            .withColumn("Cloud9am",when(col("Cloud9am")>(9),null).otherwise(col("Cloud9am")))
            .withColumn("Cloud3pm",when(col("Cloud3pm")<(0),null).otherwise(col("Cloud3pm")))
            .withColumn("Cloud3pm",when(col("Cloud3pm")>(9),null).otherwise(col("Cloud3pm")))
            .withColumn("Humidity9am",when(col("Humidity9am")<(0),null).otherwise(col("Humidity9am")))
            .withColumn("Humidity9am",when(col("Humidity9am")>(100),null).otherwise(col("Humidity9am")))
            .withColumn("Humidity3pm",when(col("Humidity3pm")<(0),null).otherwise(col("Humidity3pm")))
            .withColumn("Humidity3pm",when(col("Humidity3pm")>(100),null).otherwise(col("Humidity3pm")))
            .withColumn("Pressure9am",when(col("Pressure9am")<(970),null).otherwise(col("Pressure9am")))
            .withColumn("Pressure9am",when(col("Pressure9am")>(1050),null).otherwise(col("Pressure9am")))
            .withColumn("Pressure3pm",when(col("Pressure3pm")<(970),null).otherwise(col("Pressure3pm")))
            .withColumn("Pressure3pm",when(col("Pressure3pm")>(1050),null).otherwise(col("Pressure3pm")))
            .withColumn("WindGustSpeed",when(col("WindGustSpeed")<(0),null).otherwise(col("WindGustSpeed")))
            .withColumn("WindGustSpeed",when(col("WindGustSpeed")>(150),null).otherwise(col("WindGustSpeed")))
            .withColumn("WindSpeed9am",when(col("WindSpeed9am")<(0),null).otherwise(col("WindSpeed9am")))
            .withColumn("WindSpeed9am",when(col("WindSpeed9am")>(150),null).otherwise(col("WindSpeed9am")))
            .withColumn("WindSpeed3pm",when(col("WindSpeed3pm")<(0),null).otherwise(col("WindSpeed3pm")))
            .withColumn("WindSpeed3pm",when(col("WindSpeed3pm")>(150),null).otherwise(col("WindSpeed3pm")))
            .withColumn("Rainfall",when(col("Rainfall")<(0),null).otherwise(col("Rainfall")))
            .withColumn("Rainfall",when(col("Rainfall")>(500),null).otherwise(col("Rainfall")))
            .withColumn("Evaporation",when(col("Evaporation")<(0),null).otherwise(col("Evaporation")))
            .withColumn("Evaporation",when(col("Evaporation")>(100),null).otherwise(col("Evaporation")))
            .withColumn("Sunshine",when(col("Sunshine")<(0),null).otherwise(col("Sunshine")))
            .withColumn("Sunshine",when(col("Sunshine")>(15),null).otherwise(col("Sunshine")))

weather_org_df_ff_outl: org.apache.spark.sql.DataFrame = [Date: date, Month: int ... 23 more fields]


4.5 Scaling of the following columns in order to get correct results in Modeling 

( Note Yury code goes here )



4.5  Final out put cleaned file  - Save file

In [34]:
// Saving into Single partition
weather_org_df_ff_outl.repartition(1).write.format("com.databricks.spark.csv").option("header", "true").save("weather_s.csv")